# Метод ближайших соседей (регрессия)

Метод ближайших соседей (k Nearest Neighbors, или kNN) — простой алгоритм регрессии, основанный на оценивании сходства объектов. Суть метода проста: объект относится к тому классу, к которому принадлежит большинство из его ближайших k соседей. В данном классе реализованы классический и взвешенный KNN для регрессии.

Реализованы методы:
- fit для обучения модели
- predict для предсказания таргетов
- _init_ - конструктор
- calculate_metric - подсчитывает нужные метрики
- predict_with_weight - возвращает предсказание класса в зависимсости от weight


In [1]:
import pandas as pd
import numpy as np

In [2]:
class MyKNNReg():
    
    def __init__(self, k=3, metric='euclidean', weight='uniform'):
        '''
        Input:
        int k: the number of nearest neighbors that we will consider when defining the class. (default = 3)
        str metric: name of the metric from array ['euclidean', 'chebyshev', 'manhattan', 'cosine'] 
                                                                                        (default = 'euclidean')
        str weight: name of the metod for weighted kNN from array ['uniform', 'rank', 'distance'] 
                                                                                        (default = 'uniform')
        '''
        self.k = k
        self.metric = metric
        self.weight = weight
        # size of the training sample
        self.train_size = None
        self.X = None
        self.y = None

    
    def __str__(self):
        '''
        Output: 
        string - info about class parameters
        '''
        return f"MyKNNReg class: k={self.k}"
    
    def fit(self, X, y):
        '''
        Input:
        pd.DataFrame X: features
        pd.Series y: targets
        '''
        self.X = X
        self.y = y
        self.train_size = X.shape
        
    def predict(self, X):
        '''
        Input:
        pd.DataFrame X: features
        Output:
        np.array of predictions
        '''
        y_pred = []
        for i in range(X.shape[0]):
            distances = self.calculate_metric(X_test.iloc[i])
            distances = distances.sort_values()
            nearest_indices = distances.head(self.k).index
            nearest_classes = self.y.loc[nearest_indices]
            y_pred.append(self.predict_with_weight(nearest_classes, distances))
        return np.array(y_pred)
    
    def calculate_metric(self, x):
        '''
        Input:
        pd.Series x: vector
        Output:
        pd.Series with values of metric 
        '''
        if self.metric == 'euclidean':
            return np.sqrt(((self.X - x) ** 2).sum(axis=1))
        elif self.metric == 'chebyshev':
            return np.abs(self.X - x).max(axis=1)
        elif self.metric == 'manhattan':
            return np.abs(self.X - x).sum(axis=1)
        elif self.metric == 'cosine':
            dot_product = (self.X * x).sum(axis=1)
            norms = np.linalg.norm(self.X, axis=1) * np.linalg.norm(x)
            return 1 - dot_product / norms
    
    def predict_with_weight(self, nearest_classes, distances):
        '''
        Input:
        nearest_classes: array of classifications
        distances: values of metric
        Output:
        classification
        '''
        if self.weight == 'uniform':
            return np.mean(nearest_classes)
        
        elif self.weight == 'rank':
            weights_targets = []
            ranks = 1 / np.arange(1, self.k + 1)
            for label, rank in zip(nearest_classes.values, ranks):
                weights_targets.append(label * rank / np.sum(ranks))
            return np.sum(weights_targets)
        
        elif self.weight == 'distance':
            weights_targets = []
            distances_k = 1 / np.array(distances.head(self.k).values)
            for label, dist in zip(nearest_classes.values, distances_k):
                weights_targets.append(label * dist / np.sum(distances_k))
            return np.sum(weights_targets)

## Протестируем модель

Входные данные: датасет с различными параметрами

Выходные данные: предсказания

In [3]:
from sklearn.datasets import load_diabetes
data = load_diabetes(as_frame=True)
X, y = data['data'], data['target']


In [7]:
from sklearn.model_selection import train_test_split

model = MyKNNReg(5, 'cosine', 'rank')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
np.mean((y_pred - y_test) ** 2)

2960.7705150915244